In [23]:
from pathlib import Path
import pandas as pd
import re
from collections import Counter

In [24]:
shareholder_letters=Path('shareholder_letters.csv')

letters_df=pd.read_csv(shareholder_letters,delimiter=',')
letters_df.head(10)

,Rank,Company,Industry,Employees,"Revenue (in millions, USD)","Valuation (in millions, USD)","Profits (in millions, USD)",Profits (% of Sales),Ticker,Letter,Overflow
0,1,Walmart,General Merchandisers,"2,100,000",611289,397475.0,11680,1.9,WMT,"Dear Shareholders and Associates, Thank you. G...",NaN
1,2,Amazon.com,Internet Services and Retailing,"1,541,000",513983,1058440.0,-2722,-0.5,AMZN,Dear shareholders: As I sit down to write my s...,NaN
2,3,Exxon Mobil,Petroleum Refining,"62,000",413680,446424.0,55740,13.5,XOM,The world needs reliable and affordable energy...,NaN
3,4,Apple,"Computers, Office Equipment","164,000",394328,2609039.0,99803,25.3,AAPL,"To our shareholders,\r\nOver the past year, te...",NaN
4,5,UnitedHealth Group,Health Care: Insurance and Managed Care,"400,000",324162,440854.0,20120,6.2,UNH,"Dear Fellow Shareholders: As 2023 progresses, ...",NaN
5,6,CVS Health,Health Care: Pharmacy and Other Services,"259,500",322467,95422.0,4149,1.3,CVS,Dear Fellow Stockholders: 2022 marked a transf...,NaN
6,7,Berkshire Hathaway,Insurance: Property and Casualty (Stock),"383,000",302089,675657.0,-22819,-7.6,BRKA,To the Shareholders of Berkshire Hathaway Inc....,NaN
7,8,Alphabet,Internet Services and Retailing,"190,234",282836,1330201.0,59972,21.2,GOOGL,"To our investors,\r\n 2022 was a year full of ...",NaN
8,9,McKesson,Wholesalers: Health Care,"66,500",263966,48757.0,1114,0.4,MCK,To our valued shareholders:for McKesson as we ...,NaN
9,10,Chevron,Petroleum Refining,"43,846",246252,311093.0,35465,14.4,CVX,to our stockholders The events of 2022 demonst...,NaN


In [25]:
letters_df['ShareholderLetter']=letters_df['Letter'].fillna('')+ letters_df['Overflow'].fillna('')
letters_df= letters_df.drop(columns=['Letter', 'Overflow'])

In [26]:
def count_words(cell):
    if pd.isnull(cell):
        return pd.Series()
    else:
        words = str(cell).split()
        return pd.Series(words).value_counts()

In [27]:
letters_df['ShareholderLetter']=letters_df['ShareholderLetter'].astype(str)

In [28]:
letters_df['Letter_word_count']=[count_words(row) for row in letters_df['ShareholderLetter']]

In [29]:
stock_data=Path('../Resourses/stockdata.csv')

stock_data_df=pd.read_csv(stock_data,delimiter=',')
stock_data_df.head()

,Index,date,ticker,open,high,low,close,volume,afterHours,preMarket
0,0,2023-01-03,WMT,142.55,143.71,142.0900,143.60,5749895.0,143.60,142.04
1,1,2023-02-01,WMT,143.66,145.61,142.7000,144.67,4283059.0,144.55,143.05
2,2,2023-03-01,WMT,141.06,141.28,139.3701,140.15,4726732.0,140.02,142.16
3,3,2023-04-03,WMT,147.73,148.95,147.5000,148.69,6588882.0,148.57,147.44
4,4,2023-05-01,WMT,151.04,152.54,150.9300,151.59,3803383.0,151.59,150.99


In [30]:
stock_data_df['ticker']=stock_data_df['ticker'].apply(str)
stock_data_df.dtypes

Index           int64
date           object
ticker         object
open          float64
high          float64
low           float64
close         float64
volume        float64
afterHours    float64
preMarket     float64
dtype: object

In [31]:
ticker_list=letters_df['Ticker'].dropna()


In [32]:
openprice=[]
closeprice=[]
open_tickers=[]
close_tickers=[]
for index,row in stock_data_df.iterrows():
    stock_ticker=row['ticker']
    if row['date']=='2023-01-03':
        openprice.append(row['open'])
        open_tickers.append(stock_ticker)
    elif row['date']=='2023-12-01':
        closeprice.append(row['close'])
        close_tickers.append(stock_ticker)

In [33]:
open_df=pd.DataFrame({'Ticker':open_tickers,'OpenPrice':openprice})
close_df=pd.DataFrame({'Ticker':close_tickers,'ClosePrice':closeprice})
openclose_df=pd.merge(open_df, close_df, on='Ticker', how='left')
openclose_df['PriceChange']=openclose_df['ClosePrice']-openclose_df['OpenPrice']
openclose_df


,Ticker,OpenPrice,ClosePrice,PriceChange
0,WMT,142.55,154.34,11.79
1,AMZN,85.46,147.03,61.57
2,XOM,109.78,102.99,-6.79
3,AAPL,130.28,191.24,60.96
4,UNH,525.13,547.16,22.03
...,...,...,...,...
469,KKR,47.29,74.87,27.58
470,EQIX,663.30,824.42,161.12
471,SON,60.71,55.92,-4.79
472,NOW,395.08,690.79,295.71


In [34]:
fortune_100_df=[]
for index,row in openclose_df.iterrows():
    stock_ticker=row['Ticker'] 
    if [ticker for ticker in ticker_list]:
        fortune_100_df.append(row)
fortune_100_df=pd.DataFrame(fortune_100_df)
fortune_100_df

,Ticker,OpenPrice,ClosePrice,PriceChange
0,WMT,142.55,154.34,11.79
1,AMZN,85.46,147.03,61.57
2,XOM,109.78,102.99,-6.79
3,AAPL,130.28,191.24,60.96
4,UNH,525.13,547.16,22.03
...,...,...,...,...
469,KKR,47.29,74.87,27.58
470,EQIX,663.30,824.42,161.12
471,SON,60.71,55.92,-4.79
472,NOW,395.08,690.79,295.71


In [35]:
wordcounts_df=pd.merge(letters_df, openclose_df, on='Ticker', how='left')
wordcounts_df

,Rank,Company,Industry,Employees,"Revenue (in millions, USD)","Valuation (in millions, USD)","Profits (in millions, USD)",Profits (% of Sales),Ticker,ShareholderLetter,Letter_word_count,OpenPrice,ClosePrice,PriceChange
0,1,Walmart,General Merchandisers,"2,100,000",611289,397475.0,11680,1.9,WMT,"Dear Shareholders and Associates, Thank you. G...",and 52 to 39 our ...,142.55,154.34,11.79
1,2,Amazon.com,Internet Services and Retailing,"1,541,000",513983,1058440.0,-2722,-0.5,AMZN,Dear shareholders: As I sit down to write my s...,and 187 to 169 the ...,85.46,147.03,61.57
2,3,Exxon Mobil,Petroleum Refining,"62,000",413680,446424.0,55740,13.5,XOM,The world needs reliable and affordable energy...,and 22 our 19 to ...,109.78,102.99,-6.79
3,4,Apple,"Computers, Office Equipment","164,000",394328,2609039.0,99803,25.3,AAPL,"To our shareholders,\r\nOver the past year, te...",and 21 our 15 to ...,130.28,191.24,60.96
4,5,UnitedHealth Group,Health Care: Insurance and Managed Care,"400,000",324162,440854.0,20120,6.2,UNH,"Dear Fellow Shareholders: As 2023 progresses, ...",and 36 the 13 to ...,525.13,547.16,22.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,United Airlines Holdings,Airlines,"92,795",44955,14474.0,737,1.6,UAL,"Dear Fellow United Stockholders, On behalf of ...",the 44 to 28 and ...,38.22,40.24,2.02
96,97,Thermo Fisher Scientific,"Scientific, Photographic and Control Equipment","130,000",44915,222150.0,6950,15.5,TMO,"To our\r\nshareholders,\r\ncustomers and\r\nco...",our 79 and 64 to ...,553.29,496.13,-57.16
97,98,Qualcomm,Semiconductors and Other Electronic Components,"51,000",44200,142252.0,12936,29.3,QCOM,,"Series([], Name: count, dtype: int64)",111.35,129.67,18.32
98,99,Abbott Laboratories,Medical Products and Equipment,"115,000",43653,175984.0,6933,15.9,ABT,DEAR FELLOW SHAREHOLDER: The three years of th...,the 80 of 66 and ...,110.51,104.88,-5.63


In [36]:
newsKeyWords=Path('../Resourses/newsKeyWords.csv')

news_df=pd.read_csv(newsKeyWords,delimiter=',')
news_df

,Rank,Ticker,Company,homepages,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1,WMT,Walmart,https://www.marketwatch.com/ https://www.zacks...,investing; Cryptocurrency; Top Stories; Market...,NaN,News; Upgrades; Price Target; Reiteration; Ana...,News; Price Target; Reiteration; Analyst Ratin...,News; Analyst Color; Price Target; Reiteration...,NaN,investing; investing,investing; News; Analyst Color; Tech; Long Ide...,investing,investing,News; Price Target; Reiteration; Top Stories; ...,News; Equities; Markets; Analyst Ratings; Trad...
1,2,AMZN,Amazon.com,https://www.marketwatch.com/ https://www.zacks...,investing,investing,investing; investing,News; Earnings; After-Hours Center; Movers; Tr...,investing; News; Earnings; Penny Stocks; Large...,investing; Long Ideas; News; Analyst Color; To...,investing; News; Top Stories; Tech,investing; Cryptocurrency; Eurozone; Economics...,Asia; Economics; Markets; Tech; investing,Cryptocurrency; Asia; Economics; Federal Reser...,investing; Equities; Macro Economic Events; Br...,investing; News; Equities; Cryptocurrency; Fin...
2,3,XOM,Exxon Mobil,https://www.marketwatch.com/ https://www.zacks...,NaN,NaN,investing,investing; Long Ideas; News; Top Stories; Mark...,investing,investing,investing; Sector ETFs; Macro Economic Events;...,Sector ETFs; Large Cap; Commodities; Econ #s; ...,News; Analyst Color; Penny Stocks; Equities; P...,News; Earnings; Price Target; Intraday Update;...,Long Ideas; Sector ETFs; Short Ideas; Specialt...,NaN
3,4,AAPL,Apple,https://www.marketwatch.com/ https://www.zacks...,NaN,News; Earnings; Penny Stocks; Large Cap; Divid...,Analyst Ratings; Long Ideas; Short Ideas; Tech...,NaN,investing; News; Earnings; Penny Stocks; Large...,Long Ideas; News; Analyst Color; Top Stories; ...,investing; investing; investing,investing; Penny Stocks; Top Stories; Exclusiv...,investing; News; Earnings; Equities; Top Stori...,Cryptocurrency; Asia; Economics; Federal Reser...,Equities; Macro Economic Events; Broad U.S. Eq...,News; Analyst Color; Politics; Trading Ideas; ...
4,5,UNH,UnitedHealth Group,https://www.marketwatch.com/ https://www.zacks...,News; Health Care; After-Hours Center; Movers;...,investing,investing,investing,Analyst Ratings; investing; Upgrades; Downgrad...,NaN,investing; investing,investing,NaN,investing,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,496,KKR,KKR,https://www.marketwatch.com/ https://www.zacks...,Analyst Ratings,investing; investing,Analyst Ratings; News; Entertainment; Gaming; ...,Analyst Ratings; News; M&A; Biotech; Large Cap...,Analyst Ratings; investing; Upgrades; Downgrad...,investing; investing; investing,Analyst Ratings; investing,investing; News; M&A; Analyst Color; Earnings;...,investing; Analyst Ratings,NaN,NaN,investing
472,497,EQIX,Equinix,https://www.marketwatch.com/ https://www.zacks...,investing; investing,investing; earningscall-transcripts,Upgrades; Downgrades; Initiation; Intraday Upd...,investing,Analyst Ratings; investing,News; Analyst Color; Equities; Downgrades; Pri...,investing; investing; Upgrades; Downgrades; In...,Analyst Ratings; investing,NaN,earningscall-transcripts,NaN,investing; Analyst Ratings
473,498,SON,Sonoco Products,https://www.globenewswire.com https://www.zack...,Calendar of Events,Upgrades; Downgrades; Initiation; Intraday Upd...,Business Contracts; Long Ideas; News; Dividend...,"Environmental, Social, and Governance Criteria",NaN,NaN,Dividend Reports and Estimates; Calendar of Ev...,Product / Services Announcement; Upgrades; Dow...,Mergers and Acquisitions,Calendar of Events; Conference Calls/ Webcasts,Dividends; Specialty ETFs; New ETFs; Exclusive...,Product / Services Announcement; Product / Ser...
474,499,NOW,ServiceNow,https://www.marketwatch.com/ https://www.globe...,investing; investing; News; Earnings; Price Ta...,Contests/Awards,Product / Services Announcement,investing,Analyst Ratings,Product / Services Announcement,Product / Services Announcement; News; Price T...,Partn

In [37]:
def count_phrases(cell):
    if pd.isnull(cell):
        return pd.Series()
    else:
        phrases = str(cell).split(';')
        return pd.Series(phrases).value_counts()

In [38]:
news_df['YearWords']=news_df.loc[:,'Jan':'Dec'].apply(lambda row: ' '.join(map(str, row)), axis=1)
news_df['YearWordCount']=[count_phrases(row) for row in news_df['YearWords']]
news_df

,Rank,Ticker,Company,homepages,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,YearWords,YearWordCount
0,1,WMT,Walmart,https://www.marketwatch.com/ https://www.zacks...,investing; Cryptocurrency; Top Stories; Market...,NaN,News; Upgrades; Price Target; Reiteration; Ana...,News; Price Target; Reiteration; Analyst Ratin...,News; Analyst Color; Price Target; Reiteration...,NaN,investing; investing,investing; News; Analyst Color; Tech; Long Ide...,investing,investing,News; Price Target; Reiteration; Top Stories; ...,News; Equities; Markets; Analyst Ratings; Trad...,investing; Cryptocurrency; Top Stories; Market...,Analyst Ratings 7 Price ...
1,2,AMZN,Amazon.com,https://www.marketwatch.com/ https://www.zacks...,investing,investing,investing; investing,News; Earnings; After-Hours Center; Movers; Tr...,investing; News; Earnings; Penny Stocks; Large...,investing; Long Ideas; News; Analyst Color; To...,investing; News; Top Stories; Tech,investing; Cryptocurrency; Eurozone; Economics...,Asia; Economics; Markets; Tech; investing,Cryptocurrency; Asia; Economics; Federal Reser...,investing; Equities; Macro Economic Events; Br...,investing; News; Equities; Cryptocurrency; Fin...,investing investing investing; investing News;...,News 8 Top Storie...
2,3,XOM,Exxon Mobil,https://www.marketwatch.com/ https://www.zacks...,NaN,NaN,investing,investing; Long Ideas; News; Top Stories; Mark...,investing,investing,investing; Sector ETFs; Macro Economic Events;...,Sector ETFs; Large Cap; Commodities; Econ #s; ...,News; Analyst Color; Penny Stocks; Equities; P...,News; Earnings; Price Target; Intraday Update;...,Long Ideas; Sector ETFs; Short Ideas; Specialt...,NaN,nan nan investing investing; Long Ideas; News;...,Markets ...
3,4,AAPL,Apple,https://www.marketwatch.com/ https://www.zacks...,NaN,News; Earnings; Penny Stocks; Large Cap; Divid...,Analyst Ratings; Long Ideas; Short Ideas; Tech...,NaN,investing; News; Earnings; Penny Stocks; Large...,Long Ideas; News; Analyst Color; Top Stories; ...,investing; investing; investing,investing; Penny Stocks; Top Stories; Exclusiv...,investing; News; Earnings; Equities; Top Stori...,Cryptocurrency; Asia; Economics; Federal Reser...,Equities; Macro Economic Events; Broad U.S. Eq...,News; Analyst Color; Politics; Trading Ideas; ...,nan News; Earnings; Penny Stocks; Large Cap; D...,Top Stories 8 Economics ...
4,5,UNH,UnitedHealth Group,https://www.marketwatch.com/ https://www.zacks...,News; Health Care; After-Hours Center; Movers;...,investing,investing,investing,Analyst Ratings; investing; Upgrades; Downgrad...,NaN,investing; investing,investing,NaN,investing,NaN,NaN,News; Health Care; After-Hours Center; Movers;...,News ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,496,KKR,KKR,https://www.marketwatch.com/ https://www.zacks...,Analyst Ratings,investing; investing,Analyst Ratings; News; Entertainment; Gaming; ...,Analyst Ratings; News; M&A; Biotech; Large Cap...,Analyst Ratings; investing; Upgrades; Downgrad...,investing; investing; investing,Analyst Ratings; investing,investing; News; M&A; Analyst Color; Earnings;...,investing; Analyst Ratings,NaN,NaN,investing,Analyst Ratings investing; investing Analyst R...,News 3 inves...
472,497,EQIX,Equinix,https://www.marketwatch.com/ https://www.zacks...,investing; investing,investing; earningscall-transcripts,Upgrades; Downgrades; Initiation; Intraday Upd...,investing,Analyst Ratings; investing,News; Analyst Color; Equities; Downgrades; Pri...,investing; investing; Upgrades; Downgrades; In...,Analyst Ratings; investing,NaN,earningscall-transcripts,NaN,investing; Analyst Ratings,investing; investing investing; earningscall-t...,Analyst Ratings ...
473,498,SON,Sonoco Products,https://www.globenewswire.com https://www.zack...,Calendar of Events,Upgrades; Downgrades; Initiation; Intraday Upd...,Business Contracts; Long Ideas; News; Dividend...,"Environmental, Social, and Governance Criteria",NaN,NaN,Dividend Reports and Estimates; Calendar

In [39]:
news_count=[]
news_ticker=[]
for index,row in news_df.iterrows():
    stock_ticker=row['Ticker']
    if stock_ticker in [ticker for ticker in ticker_list]:
        news_count.append(row['YearWordCount'])
        news_ticker.append(row['Ticker']) 



In [40]:
fortune_500_df=pd.merge(news_df,openclose_df, on='Ticker', how='left')
fortune_500_df

,Rank,Ticker,Company,homepages,Jan,Feb,Mar,Apr,May,Jun,...,Aug,Sep,Oct,Nov,Dec,YearWords,YearWordCount,OpenPrice,ClosePrice,PriceChange
0,1,WMT,Walmart,https://www.marketwatch.com/ https://www.zacks...,investing; Cryptocurrency; Top Stories; Market...,NaN,News; Upgrades; Price Target; Reiteration; Ana...,News; Price Target; Reiteration; Analyst Ratin...,News; Analyst Color; Price Target; Reiteration...,NaN,...,investing; News; Analyst Color; Tech; Long Ide...,investing,investing,News; Price Target; Reiteration; Top Stories; ...,News; Equities; Markets; Analyst Ratings; Trad...,investing; Cryptocurrency; Top Stories; Market...,Analyst Ratings 7 Price ...,142.55,154.34,11.79
1,2,AMZN,Amazon.com,https://www.marketwatch.com/ https://www.zacks...,investing,investing,investing; investing,News; Earnings; After-Hours Center; Movers; Tr...,investing; News; Earnings; Penny Stocks; Large...,investing; Long Ideas; News; Analyst Color; To...,...,investing; Cryptocurrency; Eurozone; Economics...,Asia; Economics; Markets; Tech; investing,Cryptocurrency; Asia; Economics; Federal Reser...,investing; Equities; Macro Economic Events; Br...,investing; News; Equities; Cryptocurrency; Fin...,investing investing investing; investing News;...,News 8 Top Storie...,85.46,147.03,61.57
2,3,XOM,Exxon Mobil,https://www.marketwatch.com/ https://www.zacks...,NaN,NaN,investing,investing; Long Ideas; News; Top Stories; Mark...,investing,investing,...,Sector ETFs; Large Cap; Commodities; Econ #s; ...,News; Analyst Color; Penny Stocks; Equities; P...,News; Earnings; Price Target; Intraday Update;...,Long Ideas; Sector ETFs; Short Ideas; Specialt...,NaN,nan nan investing investing; Long Ideas; News;...,Markets ...,109.78,102.99,-6.79
3,4,AAPL,Apple,https://www.marketwatch.com/ https://www.zacks...,NaN,News; Earnings; Penny Stocks; Large Cap; Divid...,Analyst Ratings; Long Ideas; Short Ideas; Tech...,NaN,investing; News; Earnings; Penny Stocks; Large...,Long Ideas; News; Analyst Color; Top Stories; ...,...,investing; Penny Stocks; Top Stories; Exclusiv...,investing; News; Earnings; Equities; Top Stori...,Cryptocurrency; Asia; Economics; Federal Reser...,Equities; Macro Economic Events; Broad U.S. Eq...,News; Analyst Color; Politics; Trading Ideas; ...,nan News; Earnings; Penny Stocks; Large Cap; D...,Top Stories 8 Economics ...,130.28,191.24,60.96
4,5,UNH,UnitedHealth Group,https://www.marketwatch.com/ https://www.zacks...,News; Health Care; After-Hours Center; Movers;...,investing,investing,investing,Analyst Ratings; investing; Upgrades; Downgrad...,NaN,...,investing,NaN,investing,NaN,NaN,News; Health Care; After-Hours Center; Movers;...,News ...,525.13,547.16,22.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,496,KKR,KKR,https://www.marketwatch.com/ https://www.zacks...,Analyst Ratings,investing; investing,Analyst Ratings; News; Entertainment; Gaming; ...,Analyst Ratings; News; M&A; Biotech; Large Cap...,Analyst Ratings; investing; Upgrades; Downgrad...,investing; investing; investing,...,investing; News; M&A; Analyst Color; Earnings;...,investing; Analyst Ratings,NaN,NaN,investing,Analyst Ratings investing; investing Analyst R...,News 3 inves...,47.29,74.87,27.58
472,497,EQIX,Equinix,https://www.marketwatch.com/ https://www.zacks...,investing; investing,investing; earningscall-transcripts,Upgrades; Downgrades; Initiation; Intraday Upd...,investing,Analyst Ratings; investing,News; Analyst Color; Equities; Downgrades; Pri...,...,Analyst Ratings; investing,NaN,earningscall-transcripts,NaN,investing; Analyst Ratings,investing; investing investing; earningscall-t...,Analyst Ratings ...,663.30,824.42,161.12
473,498,SON,Sonoco Products,https://www.globenewswire.com https://www.zack...,Calendar of Events,Upgrades; Downgrades; Initiation; Intraday Upd...,Business Contracts; Long Ideas; News; Dividend...,"Environmental, Social, and Governance Criteria",NaN,NaN,...,Product / Services Announcement; Upgrades; Dow...,Mergers and

In [41]:
fortune_news_df=pd.DataFrame({'Ticker':news_ticker, 'YearWordCount':news_count})
fortune_news_df

,Ticker,YearWordCount
0,WMT,Analyst Ratings 7 Price ...
1,AMZN,News 8 Top Storie...
2,XOM,Markets ...
3,AAPL,Top Stories 8 Economics ...
4,UNH,News ...
...,...,...
89,UAL,Top Stories ...
90,TMO,Analyst Ratings ...
91,QCOM,Top Stories 7 Markets ...
92,ABT,Markets 5 Analyst Ratings...


In [42]:
fortune_100_df=pd.merge(wordcounts_df, fortune_news_df, on='Ticker', how='left')
fortune_100_df

,Rank,Company,Industry,Employees,"Revenue (in millions, USD)","Valuation (in millions, USD)","Profits (in millions, USD)",Profits (% of Sales),Ticker,ShareholderLetter,Letter_word_count,OpenPrice,ClosePrice,PriceChange,YearWordCount
0,1,Walmart,General Merchandisers,"2,100,000",611289,397475.0,11680,1.9,WMT,"Dear Shareholders and Associates, Thank you. G...",and 52 to 39 our ...,142.55,154.34,11.79,Analyst Ratings 7 Price ...
1,2,Amazon.com,Internet Services and Retailing,"1,541,000",513983,1058440.0,-2722,-0.5,AMZN,Dear shareholders: As I sit down to write my s...,and 187 to 169 the ...,85.46,147.03,61.57,News 8 Top Storie...
2,3,Exxon Mobil,Petroleum Refining,"62,000",413680,446424.0,55740,13.5,XOM,The world needs reliable and affordable energy...,and 22 our 19 to ...,109.78,102.99,-6.79,Markets ...
3,4,Apple,"Computers, Office Equipment","164,000",394328,2609039.0,99803,25.3,AAPL,"To our shareholders,\r\nOver the past year, te...",and 21 our 15 to ...,130.28,191.24,60.96,Top Stories 8 Economics ...
4,5,UnitedHealth Group,Health Care: Insurance and Managed Care,"400,000",324162,440854.0,20120,6.2,UNH,"Dear Fellow Shareholders: As 2023 progresses, ...",and 36 the 13 to ...,525.13,547.16,22.03,News ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,United Airlines Holdings,Airlines,"92,795",44955,14474.0,737,1.6,UAL,"Dear Fellow United Stockholders, On behalf of ...",the 44 to 28 and ...,38.22,40.24,2.02,Top Stories ...
96,97,Thermo Fisher Scientific,"Scientific, Photographic and Control Equipment","130,000",44915,222150.0,6950,15.5,TMO,"To our\r\nshareholders,\r\ncustomers and\r\nco...",our 79 and 64 to ...,553.29,496.13,-57.16,Analyst Ratings ...
97,98,Qualcomm,Semiconductors and Other Electronic Components,"51,000",44200,142252.0,12936,29.3,QCOM,,"Series([], Name: count, dtype: int64)",111.35,129.67,18.32,Top Stories 7 Markets ...
98,99,Abbott Laboratories,Medical Products and Equipment,"115,000",43653,175984.0,6933,15.9,ABT,DEAR FELLOW SHAREHOLDER: The three years of th...,the 80 of 66 and ...,110.51,104.88,-5.63,Markets 5 Analyst Ratings...


In [74]:
def total_word_count(df, column_name,indexcolumn,pricechangecolumn):
    indexvalues=[]
    words=[]
    counts=[]
    valuechange=[]
    total_words=[]
    
    for index, row in df.iterrows():
        word_counts = Counter(row[column_name].split())
        total_count= len(word_counts)
        for word, count in word_counts.items():
            indexvalues.append(row[indexcolumn])
            valuechange.append(row[pricechangecolumn])
            words.append(word)
            counts.append(count)
            total_words.append(total_count)
            
    word_count_df = pd.DataFrame({'Ticker':indexvalues,'Word':words, 'Count':counts,'EachWordCount':total_words,'StockPriceChange':valuechange})

    return word_count_df

In [72]:
def total_news_word_count(df, column_name,indexcolumn,pricechangecolumn):
    indexvalues=[]
    words=[]
    counts=[]
    valuechange=[]
    total_words=[]
    
    
    for index, row in df.iterrows():
        word_counts = Counter(re.split(r';\s*', row[column_name]))
        total_count= len(word_counts)
        for word, count in word_counts.items():
            indexvalues.append(row[indexcolumn])
            valuechange.append(row[pricechangecolumn])
            words.append(word)
            counts.append(count)
            total_words.append(total_count)
    word_count_df = pd.DataFrame({'Ticker':indexvalues,'Word':words, 'Count':counts,'EachWordCount':total_words,'StockPriceChange':valuechange})

    return word_count_df

In [75]:
lettertotalwordcounts_df=total_word_count(fortune_100_df,'ShareholderLetter','Ticker','PriceChange')
lettertotalwordcounts_df

,Ticker,Word,Count,EachWordCount,StockPriceChange
0,WMT,Dear,1,550,11.79
1,WMT,Shareholders,1,550,11.79
2,WMT,and,52,550,11.79
3,WMT,"Associates,",1,550,11.79
4,WMT,Thank,2,550,11.79
...,...,...,...,...,...
55312,KO,how,1,468,-4.92
55313,KO,will,1,468,-4.92
55314,KO,thrive,1,468,-4.92
55315,KO,generations,1,468,-4.92


In [77]:
def find_word_value(df,word_count,total_word_count,stock_price_change):
    wordvalue=[]
    for index,row in df.iterrows():
        value=(row[stock_price_change]/row[total_word_count])/row[word_count]
        wordvalue.append(value)
    return wordvalue

In [78]:
lettertotalwordcounts_df['WordValue']=find_word_value(lettertotalwordcounts_df,'Count','EachWordCount','StockPriceChange')

In [79]:
lettertotalwordcounts_df

,Ticker,Word,Count,EachWordCount,StockPriceChange,WordValue
0,WMT,Dear,1,550,11.79,0.021436
1,WMT,Shareholders,1,550,11.79,0.021436
2,WMT,and,52,550,11.79,0.000412
3,WMT,"Associates,",1,550,11.79,0.021436
4,WMT,Thank,2,550,11.79,0.010718
...,...,...,...,...,...,...
55312,KO,how,1,468,-4.92,-0.010513
55313,KO,will,1,468,-4.92,-0.010513
55314,KO,thrive,1,468,-4.92,-0.010513
55315,KO,generations,1,468,-4.92,-0.010513


In [80]:
newstotalwordcounts_df=total_news_word_count(fortune_500_df,'YearWords','Ticker','PriceChange')
newstotalwordcounts_df

,Ticker,Word,Count,EachWordCount,StockPriceChange
0,WMT,investing,1,30,11.79
1,WMT,Cryptocurrency,1,30,11.79
2,WMT,Top Stories,2,30,11.79
3,WMT,Markets,4,30,11.79
4,WMT,investing nan News,1,30,11.79
...,...,...,...,...,...
10772,RHI,Mid Cap,1,16,8.90
10773,RHI,Health Care,1,16,8.90
10774,RHI,Top Stories,1,16,8.90
10775,RHI,Analyst Ratings,1,16,8.90


In [82]:
newstotalwordcounts_df['WordValue']=find_word_value(newstotalwordcounts_df,'Count','EachWordCount','StockPriceChange')
newstotalwordcounts_df

,Ticker,Word,Count,EachWordCount,StockPriceChange,WordValue
0,WMT,investing,1,30,11.79,0.39300
1,WMT,Cryptocurrency,1,30,11.79,0.39300
2,WMT,Top Stories,2,30,11.79,0.19650
3,WMT,Markets,4,30,11.79,0.09825
4,WMT,investing nan News,1,30,11.79,0.39300
...,...,...,...,...,...,...
10772,RHI,Mid Cap,1,16,8.90,0.55625
10773,RHI,Health Care,1,16,8.90,0.55625
10774,RHI,Top Stories,1,16,8.90,0.55625
10775,RHI,Analyst Ratings,1,16,8.90,0.55625


Exporting DataFrames as CSV

In [ ]:

fortune_100_df.to_json('../resourses/fortune_100_final_df.json',orient='records')


In [ ]:
fortune_500_df.to_json('../resourses/fortune_500_final_df.json',orient='records')

In [ ]:
lettertotalwordcounts_df.to_json('../resourses/shareholderletter_total_word_counts.csv',orient='records')

In [ ]:
newstotalwordcounts_df.to_json('../resourses/news_total_word_counts.csv',orient='records')

: 